In [1]:
import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import ccxt
from sqlalchemy import create_engine
import matplotlib
from datetime import datetime, timedelta

with open("../api(JKBOT).txt") as f:
    lines = f.readlines()
    api_key = lines[0].strip()
    secret = lines[1].strip()

binance = ccxt.binance(config={
    'apiKey': api_key, #"XTpKrQGSk3GhXzqiEV4OfwGJzmTVcLh8dKGwHo4aQBH4p0mOqPDpIsxdh95tjGVf",
    'secret': secret, #"A0eqZGEWHsyL3NMM6WuDrucIanr7A2YZAnrwXVPhXpf2WGauIANwa5zsoNeNt0hs",
    'enableRateLimit' : True,
    'options': {
        'defaultType': 'future'
    }
})

jongmok= []

markets = binance.load_markets()
for market in markets.keys():
    if market.endswith("/USDT"):
        jongmok.append(market[:-5].lower())


def md_connect(user, password, db, host, port=3306):
    url = 'mysql+pymysql://{}:{}@{}:{}/{}'.format(user, password, host, port, db)
    engine = create_engine(url).connect()
    return engine

engine = md_connect('root', '1234', 'price_1d', 'localhost')


In [ ]:
print(jongmok[:40])

In [2]:
account = [] # 계좌 정보
diary = [] # 매매일지


window = 28
away = 7

start_date = ''
end_date = ''

tot_data = {}
engine = md_connect('root', '1234', 'price_1d', 'localhost')
for sym in jongmok[:100]:
    data = pd.read_sql(sym, engine, index_col='datetime') # 데이터 가져오기
    tot_data[sym] = data

In [47]:
from tensorflow import keras
model = keras.Sequential()
model.add(keras.layers.LSTM(30, activation='relu', input_shape=(window, 4), dropout=0.3))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.load_weights('h5/1d_14_7_(30).h5')

engine = md_connect('root', '1234', 'ind_test', 'localhost')


for key in jongmok[:80]:
    data = tot_data[key]
    x_data = tot_data[key][['open', 'high', 'low', 'close']]
    y_data = tot_data[key][['close']]

    x_input = []
    y_input = []

    ## 전처리
    for i in range(len(data)-window-away+1): #len(data)-window
        mean = x_data[i:i+window].mean()
        std = x_data[i:i+window].std()
        x = (x_data[i:i+window] - mean) / std
        x_input.append(x)
    #
    ## ind 지표 생성
    x_input = np.array(x_input)
    predict_data = model.predict(x_input).tolist()

    ## 수익률 생성
    data['rate'] = data['close'].pct_change(periods=away).shift(-away)


    predict = []
    for i in range(len(predict_data)):
        predict.append(predict_data[i][0])
    data = data[window-1:-away]

    ##ind 지표 결합
    data = data.copy()
    data['ind'] = predict
    print(len(data))

    data['sym'] = key

    data = data[['sym', 'open', 'high', 'low', 'close', 'rate', 'ind']]

    data.to_sql(key, engine, if_exists="replace", index=True)

40/40 [==============================] - 0s 957us/step
1262
37/37 [==============================] - 0s 861us/step
1182
37/37 [==============================] - 0s 889us/step
1160
36/36 [==============================] - 0s 800us/step
1142
36/36 [==============================] - 0s 829us/step
1140
36/36 [==============================] - 0s 857us/step
1139
36/36 [==============================] - 0s 829us/step
1133
36/36 [==============================] - 0s 872us/step
1132
36/36 [==============================] - 0s 886us/step
1131
36/36 [==============================] - 0s 1ms/step
1128
35/35 [==============================] - 0s 971us/step
1117
35/35 [==============================] - 0s 971us/step
1114
35/35 [==============================] - 0s 824us/step
1113
35/35 [==============================] - 0s 824us/step
1112
35/35 [==============================] - 0s 883us/step
1111
35/35 [==============================] - 0s 824us/step
1107
35/35 [==============================] - 0

In [3]:
engine = md_connect('root', '1234', 'ind_test', 'localhost')
for sym in jongmok[:80]:
    data = pd.read_sql(sym, engine, index_col='datetime') # 데이터 가져오기
    tot_data[sym] = data

# 변수 설정
ind_data = []
for key in jongmok[:80]:
    ind_data.append(tot_data[key][['sym', 'open', 'high', 'low', 'close', 'rate', 'ind']])

ind_data = pd.concat(ind_data)


def select_rate(long_std, short_std, start_date=0, end_date=0, fee=0):
    df = ind_data[(ind_data['ind'] > long_std) | (ind_data['ind'] < short_std)].sort_index()
    condition = df['ind'] > 0.5
    df = df.assign(position=condition.map({True:1, False:-1}))
    df['profit'] = df['position'] * df['rate'] -(2 * fee)

    if start_date != 0:
        if end_date != 0:
            df = df.loc[start_date:end_date]
        else:
            df = df.loc[start_date:]
    else:
        if end_date !=0:
            df = df.loc[:end_date]


    df['count_sum'] = df.groupby('datetime').count()['sym'].rolling(window=7, min_periods=1).sum()
    df['tot_profit'] = df['profit'].cumsum()

    return df
log_column = ['시작', '끝', '거래기간', '매수조건', '매도조건', '거래 횟수', '승률', '승수', '패수', '평균보유량', '평균수익률', '총수익률']
log_df = pd.DataFrame(columns = log_column)
import matplotlib.pyplot as plt

In [4]:
log_df

,시작,끝,거래기간,매수조건,매도조건,거래 횟수,승률,승수,패수,평균보유량,평균수익률,총수익률


In [5]:
ind_data

,sym,open,high,low,close,rate,ind
datetime,,,,,,,
2019-10-05,btc,8130.0000,8175.0000,8010.0000,8122.1400,0.021925,0.457285
2019-10-06,btc,8122.0100,8148.4300,7760.0000,7852.7900,0.053722,0.459272
2019-10-07,btc,7851.0100,8330.0000,7764.4200,8186.8800,0.019442,0.456518
2019-10-08,btc,8188.5500,8322.0100,8103.8800,8167.6200,-0.000932,0.478046
2019-10-09,btc,8168.3300,8700.0000,8113.0000,8559.7700,-0.066306,0.500909
...,...,...,...,...,...,...,...
2023-03-08,sand,0.6092,0.6125,0.5600,0.5686,0.028843,0.351274
2023-03-09,sand,0.5686,0.5896,0.5120,0.5336,0.119565,0.418443
2023-03-10,sand,0.5336,0.5397,0.5035,0.5366,0.264443,0.439132


In [ ]:
start_data = '2021-01-01'

In [8]:
start_data = '2022-02-01'
start_date = datetime.strptime(start_data, '%Y-%m-%d')
ind_data.loc[start_date:]
long = len(ind_data.loc[start_date][ind_data.loc[start_date]['ind'] > 0.5])
short = len(ind_data.loc[start_date][ind_data.loc[start_date]['ind'] < 0.5])
print(long, short)
ind_data.loc[start_date]

KeyError: 'Cannot get left slice bound for non-unique label: datetime.datetime(2022, 2, 1, 0, 0)'

In [ ]:
start_data = '2021-03-01'
start_date = datetime.strptime(start_data, '%Y-%m-%d')

In [ ]:
ind_data[ind_data.loc[start_date]['ind'] > 0.5]

In [9]:
start_data = '2023-01-01'
start_date = datetime.strptime(start_data, '%Y-%m-%d')
new_ind_list = []
for i in range(365):
    start_date
    long = len(ind_data.loc[start_date][ind_data.loc[start_date]['ind'] > 0.5])
    short = len(ind_data.loc[start_date][ind_data.loc[start_date]['ind'] < 0.5])
    new_ind_list.append(long / (long + short))
    print(start_date, long, short, long / (long + short))
    start_date = start_date + timedelta(days=1)

2023-01-01 00:00:00 65 14 0.8227848101265823
2023-01-02 00:00:00 68 11 0.8607594936708861
2023-01-03 00:00:00 72 7 0.9113924050632911
2023-01-04 00:00:00 73 6 0.9240506329113924
2023-01-05 00:00:00 70 9 0.8860759493670886
2023-01-06 00:00:00 66 13 0.8354430379746836
2023-01-07 00:00:00 57 22 0.7215189873417721
2023-01-08 00:00:00 51 28 0.6455696202531646
2023-01-09 00:00:00 57 22 0.7215189873417721
2023-01-10 00:00:00 63 16 0.7974683544303798
2023-01-11 00:00:00 63 16 0.7974683544303798
2023-01-12 00:00:00 67 12 0.8481012658227848
2023-01-13 00:00:00 69 10 0.8734177215189873
2023-01-14 00:00:00 70 9 0.8860759493670886
2023-01-15 00:00:00 71 8 0.8987341772151899
2023-01-16 00:00:00 69 10 0.8734177215189873
2023-01-17 00:00:00 59 20 0.7468354430379747
2023-01-18 00:00:00 51 28 0.6455696202531646
2023-01-19 00:00:00 37 42 0.46835443037974683
2023-01-20 00:00:00 16 63 0.20253164556962025
2023-01-21 00:00:00 7 72 0.08860759493670886
2023-01-22 00:00:00 9 70 0.11392405063291139
2023-01-23 00

KeyError: datetime.datetime(2023, 3, 13, 0, 0)

In [ ]:
ind_data['ascending'] = abs(ind_data['ind'] - 0.5)+0.5
ind_data = ind_data.sort_values(['datetime', 'ascending'], ascending=[True, False])

In [ ]:
start_date = '2023-01-01'
start_date = datetime.strptime(start_date, '%Y-%m-%d')
new_ind_list = pd.DataFrame([], columns=ind_data.columns.tolist())
for i in range(360):
    start_date
    # long = len(ind_data.loc[start_date][ind_data.loc[start_date]['ind'] > 0.5])
    # short = len(ind_data.loc[start_date][ind_data.loc[start_date]['ind'] < 0.5])
    new_ind_list = pd.concat([new_ind_list, ind_data.loc[start_date][:5]])
    start_date = start_date + timedelta(days=1)

In [ ]:
df = new_ind_list

condition = df['ind'] > 0.5
df = df.assign(position=condition.map({True:1, False:-1}))
df['profit'] = df['position'] * df['rate']
df['tot_profit'] = df['profit'].cumsum()
win_cnt = (df['profit'] > 0).sum()
lose_cnt = (df['profit'] < 0).sum()
df['profit'].mean()

In [ ]:
win_cnt / (win_cnt + lose_cnt)

In [ ]:
df

In [57]:
log_column = ['시작', '끝', '거래기간', '매수조건', '매도조건', '거래 횟수', '승률', '승수', '패수', '평균보유량', '평균수익률', '총수익률']
log_df = pd.DataFrame(columns = log_column)
import matplotlib.pyplot as plt

In [58]:
start_date = '2023-01-01'
start_date = datetime.strptime(start_date, '%Y-%m-%d')
end_date = start_date + timedelta(days=30)

for i in range(36):
    long_std = 0.75
    short_std = 0.25

    df = select_rate(long_std, short_std, start_date, end_date, 0.0004)

    win_cnt = (df['profit'] > 0).sum()
    lose_cnt = (df['profit'] < 0).sum()

    day = end_date - start_date
    day.days


    # log.append([len(df), day.days, win_cnt / len(df), win_cnt, lose_cnt, df['count_sum'].mean(), df['profit'].mean(), df['profit'].sum()])
    log = {'시작' : start_date, '끝' : end_date, 'window' : window, 'away' : away, '매수조건' : long_std, '매도조건' : short_std, '거래 횟수' : len(df), '거래기간' : day.days, '승률' : win_cnt / len(df), '승수' : win_cnt, '패수' : lose_cnt, '평균보유량' : df['count_sum'].mean(), '평균수익률' : df['profit'].mean(), '총수익률' : df['profit'].sum()}
    log_df = log_df.append(log, ignore_index=True)

    start_date = end_date + timedelta(days=1)
    end_date = end_date + timedelta(days=31)


C:\Users\tomkj\AppData\Local\Temp\ipykernel_5792\2913601241.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_df = log_df.append(log, ignore_index=True)
C:\Users\tomkj\AppData\Local\Temp\ipykernel_5792\2913601241.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_df = log_df.append(log, ignore_index=True)
C:\Users\tomkj\AppData\Local\Temp\ipykernel_5792\2913601241.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_df = log_df.append(log, ignore_index=True)
C:\Users\tomkj\AppData\Local\Temp\ipykernel_5792\2913601241.py:19: RuntimeWarning: invalid value encountered in longlong_scalars
  log = {'시작' : start_date, '끝' : end_date, 'window' : window, 'away' : away, '매수조건' : long_std, '매도조건' : short_std, '거래 횟수'

In [59]:
log_df

,시작,끝,거래기간,매수조건,매도조건,거래 횟수,승률,승수,패수,평균보유량,평균수익률,총수익률,away,window
0,2023-01-01,2023-01-31,30,0.75,0.25,9,0.555556,5,4,5.333333,0.004355,0.039197,7.0,14.0
1,2023-02-01,2023-03-03,30,0.75,0.25,114,0.850877,97,17,35.578947,0.050474,5.754045,7.0,14.0
2,2023-03-04,2023-04-03,30,0.75,0.25,5,0.000000,0,5,5.000000,-0.093681,-0.468407,7.0,14.0
3,2023-04-04,2023-05-04,30,0.75,0.25,0,NaN,0,0,NaN,NaN,0.000000,7.0,14.0
4,2023-05-05,2023-06-04,30,0.75,0.25,0,NaN,0,0,NaN,NaN,0.000000,7.0,14.0
5,2023-06-05,2023-07-05,30,0.75,0.25,0,NaN,0,0,NaN,NaN,0.000000,7.0,14.0
6,2023-07-06,2023-08-05,30,0.75,0.25,0,NaN,0,0,NaN,NaN,0.000000,7.0,14.0
7,2023-08-06,2023-09-05,30,0.75,0.25,0,NaN,0,0,NaN,NaN,0.000000,7.0,14.0
8,2023-09-06,2023-10-06,30,0.75,0.25,0,NaN,0,0,NaN,NaN,0.000000,7.0,14.0
9,2023-10-07,2023-11-06,30,0.75,0.25,0,NaN,0,0,NaN,NaN,0.000000,7.0,14.0


In [ ]:
ind_data

In [ ]:
test_df = ind_data[ind_data['sym'] == 'eth']

In [ ]:
test_df

In [ ]:
condition = test_df['ind'] > 0.8

In [ ]:
matching_rows = test_df[condition].index.tolist()

In [ ]:
rows_to_keep = []
chart_df = {}
for date in matching_rows:
    start_date = date - pd.Timedelta(days=28)
    end_date = date + pd.Timedelta(days=6)
    chart_df[date] = test_df.loc[start_date:end_date]

In [ ]:
list = []
for key in chart_df:
    list.append(key)

In [ ]:
i=0

In [ ]:
import mplfinance as mpf
print(list[i])
print(chart_df[list[i]]['rate'].loc[list[i]])
mpf.plot(chart_df[list[i]], type='candle')
i += 1

In [ ]:
import matplotlib.pyplot as plt
import mplfinance as mpf
#
# print(list[i])
# print(chart_df[list[i]]['rate'].loc[list[i]])
fig = plt.figure(figsize=(14,10))
ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 1, 2)
ax1.plot(test_df['close'][:100], label='close')
ax2.plot(test_df['new'][:100], label='tot_profit')
plt.show()
i += 1

In [ ]:
chart_df[list[0]]